In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

#import utils 
#reloads modules automagically, changes
#to code update without having to close the notebook
%load_ext autoreload
%autoreload 2

import os

from bs4 import BeautifulSoup
import blooms_preprocess_utils
import textstat

from snorkel.labeling import labeling_function
from snorkel.labeling.model import LabelModel


from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
label_model = LabelModel(cardinality=2, verbose=True)
blooms_labeler = label_model.load('./best_label_model.pkl')

In [ ]:
X_train_weak = pd.read_csv('./vectorizer_data/X_train_weak.csv', index_col=0)
X_train_weak.head()

,text
0,Define selectively permeable in your own words.
1,Why did the Spanish build their colonies along...
2,A curved mirror produces a virtual image that ...
4,When closely related individuals mate with eac...
5,Define interstitial cell of Leydig in your own...


In [ ]:
X_train_weak = pd.read_csv('./vectorizer_data/X_train_weak.csv', index_col=0)['text']
y_train_weak = pd.read_csv('./vectorizer_data/y_train_weak.csv', index_col=0)

X_train = pd.read_csv('./vectorizer_data/X_train.csv', index_col=0)['text']
y_train = pd.read_csv('./vectorizer_data/y_train.csv', index_col=0)

X_test = pd.read_csv('./vectorizer_data/X_test.csv', index_col=0)['text']
y_test = pd.read_csv('./vectorizer_data/y_test.csv', index_col=0)


v = TfidfVectorizer()

x_train_weak = v.fit_transform(X_train_weak.values.astype('U'))
x_test_weak = v.transform(X_test.values.astype('U'))

v = TfidfVectorizer()

x_train = v.fit_transform(X_train.values.astype('U'))
x_test = v.transform(X_test.values.astype('U'))


lin_svm_supervised = SVC(kernel="linear", C=0.025)
lin_svm_supervised.fit(x_train, y_train);

#### weak supervision portion
lin_svm_weak = SVC(kernel="linear", C=0.025)
lin_svm_weak.fit(x_train_weak, y_train_weak);



/root/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(lin_svm_weak.coef_.shape[-1])
lin_svm_supervised.coef_.shape[-1]


14099


14829

## Predicting labels

- go through questions
- show label activation

## Loading models

SKlearn model weights are currently saved in 2 pickle files, we can load them easily with a couple lines of code as shown below:

In [ ]:
# with open("supervised_end_model_weights.pkl", 'rb') as file:
#     supervised_model = pickle.load(file)
# with open("weak_end_model_weights.pkl", 'rb') as file:
#     weak_model = pickle.load(file)

## Question prediction

We have 10 example questions, which we will predict labels for using both the weakly supervised model and the fully supervised model.

The labels correspond to 0 for Lower Order (), and 1 for High Order ().

First, however, we have to make sure we extract the correct features from the example questions.

will demonstrate (capabilities or weak supervision?)

In [ ]:
q_idx_list = [3064, 2293, 444, #(weak wrong) 
764, 62, 3454, 1929, 1244, 1830, #(both wrong),
 3074]

Explain why the intensity of a sound is proportional to its amplitude squared.

Define product rule in your own words.

Which of the following statements about epigenetic regulation is false?


The conservation movement of the early twentieth century echoed the progressive movement’s emphasis on


Sometimes it is necessary to take a PET scan very soon after ingesting a radiopharmaceutical. Why is that the case?


What kind of heat transfer requires no medium?


How did the British use the institution of slavery as a tool against the colonists in the Revolutionary War?

Define apomixis in your own words.


Biodiversity decreases when moving away from the _________.


Historians looking to study the role of African rulers and merchants in the Atlantic slave trade would likely consult..

In [ ]:
X_test.iloc[q_idx_list[1]]

'Define product rule in your own words.'

In [ ]:
X_test.iloc[q_idx_list[2]]

'Which of the following statements about epigenetic regulation is false?'

In [ ]:
X_test.iloc[q_idx_list[3]]

'The conservation movement of the early twentieth century echoed the progressive movement’s emphasis on'

In [ ]:
X_test.iloc[q_idx_list[4]]

'Sometimes it is necessary to take a PET scan very soon after ingesting a radiopharmaceutical. Why is that the case?'

In [ ]:
X_test.iloc[q_idx_list[5]]

'What kind of heat transfer requires no medium?'

In [ ]:
X_test.iloc[q_idx_list[6]]

'How did the British use the institution of slavery as a tool against the colonists in the Revolutionary War?'

In [ ]:
X_test.iloc[q_idx_list[7]]

'Define apomixis in your own words.'

In [ ]:
X_test.iloc[q_idx_list[8]]

'Biodiversity decreases when moving away from the _________.'

In [ ]:
X_test.iloc[q_idx_list[9]]

'Historians looking to study the role of African rulers and merchants in the Atlantic slave trade would likely consult'

ok let's just find questions and look up the text in the test dataframes.. so, now: we search in the X_test dataframe, then match indices in the other ones..

In [ ]:
df = pd.read_csv('questions_utf8.csv')


mapping = {1:1, 2:2, 3:2, 4:2, 5:2, 6:2}
preprocess_df = blooms_preprocess_utils.preprocess_dataset(df, mapping);

readability = []
for i in range(len(preprocess_df)):
    readability.append(textstat.textstat.flesch_reading_ease(preprocess_df.iloc[i].text))


/work/edu_research/big_dataset/blooms_preprocess_utils.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  'k12phys': 'physics', 'inbook-yes__book:': 'not_sure'}
/work/edu_research/big_dataset/blooms_preprocess_utils.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['blooms'] = df['blooms'].map(class_mapping)
/work/edu_research/big_dataset/blooms_preprocess_utils.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [ ]:
t_df = X_test.reset_index(drop = True)
t_df.sample(8)

2058    What is 32.0^{\circ}\text{C} in degrees Fahren...
3240    Mammals, reptiles, and birds have the followin...
2827    \nAfter traveling along a path, a toy car’s fi...
2923    The slogan “Fifty-four Forty or Fight” best de...
778     Which group would most likely oppose a U.S. de...
2950                       Define lens in your own words.
1881    \nThe levels of carbon dioxide levels increase...
2159    Joints are classified based on the material co...
Name: text, dtype: object

In [ ]:
q_idx = 2159

display(preprocess_df[preprocess_df.text == X_test.iloc[q_idx]])

print(preprocess_df[preprocess_df.text == X_test.iloc[q_idx]]['type'])


supervised_label = lin_svm_supervised.predict(x_test[q_idx, :])
weak_label = lin_svm_weak.predict(x_test_weak[q_idx, :])

print("blooms label:",
 preprocess_df[preprocess_df.text == X_test.iloc[q_idx]]['adj_label'].values)
print("supervised model predicts:", supervised_label)
print("weak model predicts:", weak_label)
print(X_test.iloc[q_idx])



,uid,uuid,group_uuid,published_at,cnxmod,question,blooms,book,type,subject,adj_label,glossary_terms,text,text_length,num_words
13547,9899@6,e4069425-9d47-498c-9496-d4a4fe465eb0,10262471-b1e0-418b-a6a5-10a04e0bcb89,2019-07-01T15:09:57.888Z,f3a5c924-6b1b-4d1b-b506-906b00da0be5__d31fc440...,Joints are classified based on the material co...,2,bio__apbio,__conceptual-or-recall,biology,1,1.0,Joints are classified based on the material co...,164,29


13547    __conceptual-or-recall
Name: type, dtype: object
blooms label: [1]
supervised model predicts: [1]
weak model predicts: [1]
Joints are classified based on the material composing the joint and the presence or absence of a cavity in the joint. Which of the following are the weakest joints?


In [ ]:
q_idx_list = [1867, 2293, 444, #(weak wrong)
764, 1013, 2159, 1929, 1244, 1830, #(both wrong),
 3074]

# q_idx_list = [1867, 2293, 444, 
# # 764, 1013, 1929, 1244, 1830, #(both wrong),
# #  3074]

uid_list = []
for i in range(len(q_idx_list)):
    uid = preprocess_df[preprocess_df.text == X_test.iloc[q_idx_list[i]]].uid
    text = preprocess_df[preprocess_df.text == X_test.iloc[q_idx_list[i]]].text
    type = preprocess_df[preprocess_df.text == X_test.iloc[q_idx_list[i]]]['type']

    print(i)
    print(q_idx_list[i])
    print(text.values[0])
    print(type.values)

    print(df[df.uid == uid.values[0]].blooms)
    print(df[df.uid == uid.values[0]].)
    print('-'*50)

    uid_list.append([uid.values[0], text.values[0]])



0
1867
What is the principle function or role of triglycerides in animals?
['__conceptual-or-recall']
6609    2
Name: blooms, dtype: int64
--------------------------------------------------
1
2293
Define product rule in your own words.
['recall']
4465    1
Name: blooms, dtype: int64
--------------------------------------------------
2
444
Which of the following statements about epigenetic regulation is false?
['concept-coach__conceptual-or-recall'
 'concept-coach__conceptual-or-recall' '__conceptual-or-recall']
12370    3
Name: blooms, dtype: int64
--------------------------------------------------
3
764
The conservation movement of the early twentieth century echoed the progressive movement’s emphasis on
['conceptual']
17089    4
Name: blooms, dtype: int64
--------------------------------------------------
4
1013
Why is carbon considered a macronutrient?
['concept-coach__conceptual-or-recall']
9249    2
Name: blooms, dtype: int64
--------------------------------------------------
5
21

In [ ]:
supervised_labels= lin_svm_supervised.predict(x_test)

weak_labels = lin_svm_weak.predict(x_test_weak)

supervised_labels.shape


(3544,)

In [ ]:
## predict on questions
supervised_labels = supervised_model.predict()

weak_labels = weak_model.predict()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bcbe6184-b36a-4097-968c-a2ee6ebad722' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>